In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89300,3.0,0.0,2.0,3.0,2.0,3.0,13.0,8.0,9.0,9.0,...,6.0,5.0,3.0,1.0,6.0,1.0,3.0,2022-12-27,265,3.0
89301,6.0,4.0,0.0,0.0,2.0,0.0,14.0,7.0,8.0,4.0,...,4.0,2.0,1.0,2.0,2.0,2.0,8.0,2022-12-28,265,1.0
89302,7.0,2.0,3.0,4.0,7.0,4.0,10.0,9.0,7.0,11.0,...,2.0,3.0,5.0,1.0,1.0,0.0,8.0,2022-12-29,265,3.0
89303,6.0,5.0,4.0,3.0,0.0,3.0,11.0,12.0,9.0,10.0,...,3.0,3.0,1.0,2.0,0.0,1.0,2.0,2022-12-30,265,7.0


In [7]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 10, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(64925, 674)
y_train.shape=(64925,)
X_test.shape=(24380, 674)
y_test.shape=(24380,)


In [8]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [9]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2023-12-30 13:29:33,831] A new study created in memory with name: no-name-a2d67f29-e46f-48d0-bd34-c5a8b65c094a
[I 2023-12-30 13:29:45,973] Trial 0 finished with value: 3.3949475786451973 and parameters: {'num_leaves': 199, 'feature_fraction': 0.2121822406519419, 'bagging_fraction': 0.625980871198003, 'min_child_samples': 81}. Best is trial 0 with value: 3.3949475786451973.
[I 2023-12-30 13:30:00,601] Trial 1 finished with value: 3.304676449279895 and parameters: {'num_leaves': 224, 'feature_fraction': 0.3946502766387203, 'bagging_fraction': 0.2580649936869648, 'min_child_samples': 29}. Best is trial 1 with value: 3.304676449279895.
[I 2023-12-30 13:30:16,816] Trial 2 finished with value: 3.2859018423936885 and parameters: {'num_leaves': 208, 'feature_fraction': 0.955665344294186, 'bagging_fraction': 0.27592934084299703, 'min_child_samples': 31}. Best is trial 2 with value: 3.2859018423936885.
[I 2023-12-30 13:30:25,379] Trial 3 finished with value: 3.272526823163145 and parameters: 

In [10]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 79, 'feature_fraction': 0.45396627774368803, 'bagging_fraction': 0.5808068482240976, 'min_child_samples': 45}


In [11]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.45396627774368803, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45396627774368803
[LightGBM] [Warning] bagging_fraction is set=0.5808068482240976, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5808068482240976
[LightGBM] [Warning] feature_fraction is set=0.45396627774368803, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45396627774368803
[LightGBM] [Warning] bagging_fraction is set=0.5808068482240976, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5808068482240976
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.069092 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 161809
[LightGBM] [Info] Number of data points in the train set: 64925, number of used features: 675
[LightGBM] [Info] Start training from 

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x1359c11f0>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.5808068482240976,
                               feature_fraction=0.45396627774368803,
                               min_child_samples=45, num_leaves=79))])

In [12]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.45396627774368803, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45396627774368803
[LightGBM] [Warning] bagging_fraction is set=0.5808068482240976, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5808068482240976
test_mae=2.6088


In [13]:
from src.plot import plot_one_sample

plot_one_sample(
    example_id=2979,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

In [14]:
plot_one_sample(
    example_id=4979,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)